In [5]:
import pandas as pd
import requests
import numpy as np

In [9]:
# Codigo para Media dos Mais Caros e Mais Baratos ANP em SP e RJ
# Realizado: Media ponderada para cada combustivel de cada estado

#def get_anp_data():
comb = ['diesel_rj', 'diesel_sp', 'diesel_s10_rj', 'diesel_s10_sp', 
        'etanol_rj', 'etanol_sp', 'gasol_adit_rj', 'gasol_adit_sp', 
        'gasolina_sp', 'gasolina_rj', 'glp_rj', 'glp_sp', 'gnv_rj', 'gnv_sp']

data = {'Estado': ['RJ', 'RJ', 'SP', 'SP'], 'Tipo': ['Mais Caro', 'Mais Barato', 'Mais Caro', 'Mais Barato'], 'Diesel': [], 'Diesel S10': [], 'Etanol': [], 'Gasolina Adit': [], 'Gasolina': [], 'GLP': [], 'GNV': []}
data_keys = list(data.keys())

url = 'https://www.primetalkdata.com/datacenter/combustiveis/'

for type in ['mais_caro/', 'mais_barato/']:
        date = requests.get(url + type).text.split('<a href="')[-1][:10]

        for idx in range(len(comb)):
                auxDF = pd.DataFrame([elem.split(',') for elem in requests.get(url + type + date + '_' + comb[idx] + '.csv').text.split('\r\n')])
                auxDF.columns = auxDF.iloc[0]
                auxDF.drop(auxDF.index[0], inplace=True)
                auxDF.dropna(inplace=True)

                val = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-1:].astype(float).values
                wgt = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-3:-2].astype(int).values

                data[data_keys[2 + idx//2]].append(round(np.average(val, weights=wgt), 2))

#result = pd.DataFrame(data, index=pd.MultiIndex.from_tuples([('RJ', 'Mais Caro'), ('RJ', 'Mais Barato'), ('SP', 'Mais Caro'), ('SP', 'Mais Barato')]))
a = pd.DataFrame(data)
a


,Estado,Tipo,Diesel,Diesel S10,Etanol,Gasolina Adit,Gasolina,GLP,GNV
0,RJ,Mais Caro,6.41,6.69,4.79,6.40,6.05,116.93,6.04
1,RJ,Mais Barato,6.49,6.61,3.99,6.43,6.33,114.55,4.83
2,SP,Mais Caro,5.75,5.87,3.86,5.56,5.28,85.01,4.26
3,SP,Mais Barato,5.67,5.88,3.06,5.44,5.48,85.56,3.90


In [13]:
a.transpose().reset_index().rename(columns={'index':'x', 0: 'y', 1: 'v2', 2: 'v3', 3: 'v4'})

,x,y,v2,v3,v4
0,Estado,RJ,RJ,SP,SP
1,Tipo,Mais Caro,Mais Barato,Mais Caro,Mais Barato
2,Diesel,6.41,6.49,5.75,5.67
3,Diesel S10,6.69,6.61,5.87,5.88
4,Etanol,4.79,3.99,3.86,3.06
5,Gasolina Adit,6.4,6.43,5.56,5.44
6,Gasolina,6.05,6.33,5.28,5.48
7,GLP,116.93,114.55,85.01,85.56
8,GNV,6.04,4.83,4.26,3.9


In [15]:
# Codigo para 5 Maiores Altas e 5 Maiores Quedas pela CEAGESP
# Realizado: 5 maiores altas e baixas percentuais entre todos os setores 

url = 'https://www.primetalkdata.com/datacenter/ceagesp/2023/'
date = requests.get(url).text.split('<a href="')[-1][:10]

data = {'Tipo': [], 'Setor': [], 'Variação': [], 'Item': [], 'Percentual': []}

for type in ['alta', 'baixa']:
    resp = requests.get(url + date + '_ceagesp.csv')
    resp.encoding = resp.apparent_encoding
    auxDF = pd.DataFrame([elem.split(',') for elem in resp.text.split('\r\n')])
    auxDF.columns = auxDF.iloc[0]
    auxDF.drop(auxDF.index[0], inplace=True)
    auxDF.dropna(inplace=True)

    auxDF['pct_' + type] = pd.to_numeric(auxDF['pct_' + type], errors='coerce')
    auxDF = auxDF.sort_values('pct_' + type, ascending=(type=='baixa')).head(5)
    
    data['Tipo'].extend(['Maiores Altas']*5 if type == 'alta' else ['Maiores Baixas']*5) 
    data['Setor'].extend(auxDF.setor.values)
    data['Variação'].extend(auxDF.variacao.values)
    data['Item'].extend(auxDF['top_' + type + 's'].values)
    data['Percentual'].extend(auxDF['pct_' + type].values) 

pd.DataFrame(data).transpose().reset_index().rename(columns={'index':'x', 0: 'y', 1: 'v2', 2: 'v3', 3: 'v4', 4: 'v5', 5: 'v6', 6: 'v7', 7: 'v8', 8: 'v9', 9: 'v10' })

,x,y,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,Tipo,Maiores Altas,Maiores Altas,Maiores Altas,Maiores Altas,Maiores Altas,Maiores Baixas,Maiores Baixas,Maiores Baixas,Maiores Baixas,Maiores Baixas
1,Setor,VERDURAS,LEGUMES,FRUTAS,VERDURAS,VERDURAS,PESCADOS,VERDURAS,VERDURAS,LEGUMES,PESCADOS
2,Variação,11.33,16.20,2.96,17.25,21.15,-10.03,-10.21,-13.00,-0.88,-10.03
3,Item,SALSA,TOMATE CAQUI,LIMÃO TAITI,RABANETE,COENTRO,SARDINHA LAGES,COENTRO,SALSA,PIMENTÃO AMARELO,PEROÁ BRANCO
4,Percentual,122.15,84.03,74.47,71.15,70.91,-58.89,-56.97,-53.5,-49.7,-48.7
